---
title: Audio processing pipeline
output-file: audio_processor.html
description: Loads raw audio streams
---

In [ ]:
#| default_exp audio_processor

In [ ]:
#| export

from glob import glob
import os
from joblib import Parallel, delayed
from pathlib import Path
from typing import List
from pydub import AudioSegment
import numpy as np

def chunk_audio(audio: AudioSegment, increment_mills: int = 300, windows: int = 1000) -> List[AudioSegment]:
    sample_rate = audio.frame_count(ms=1000)
    seconds = audio.frame_count()/sample_rate
    def get_segment(e, window):
        return audio[e-windows:e]
    audio_segments = Parallel(n_jobs=4)(delayed(get_segment)(e, windows) for e in np.arange(windows, seconds*1000, increment_mills))
    return audio_segments

def export_audio(segment: AudioSegment, path: str):
    return segment.export(f'{path}.wav', format="wav")

def chunk_selected_audios(audios: List[str], output_path: str) -> None:
    Path(output_path).mkdir(parents=True, exist_ok=True)
    for file in audios:
        audio = AudioSegment.from_wav(file)
        for ix, segment in enumerate(chunk_audio(audio)):
            path = os.path.join(output_path, f"{Path(file).stem}_{ix}.wav")
            export_audio(segment, path)

In [ ]:
#| export

import shutil
from typing import List
from collections import Counter
from transformers import AutoFeatureExtractor
from transformers import Wav2Vec2ForXVector, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
import torch
from numpy.typing import NDArray
from pydantic import BaseModel
from io import BytesIO
from glob import glob
from random import choices

class AudioArray(BaseModel):
    array: np.ndarray
    class Config:
        arbitrary_types_allowed = True

class AudioProcessor(object):

    def __init__(self, model_checkpoint: str) -> None:
        self.model_checkpoint = model_checkpoint
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(self.model_checkpoint)
    
    def preprocess_function(self, audio_arrays: List[AudioArray], max_duration: float = 1.0):
        arrays = [a.array for a in audio_arrays]
        inputs = self.feature_extractor(
            arrays,
            sampling_rate=self.feature_extractor.sampling_rate, 
            max_length=int(self.feature_extractor.sampling_rate * max_duration),
            truncation=True,
            return_tensors="pt",
            )
        return inputs
    
    def parse_audio(self, raw_data: bytes, sample_width: int, channels: int, frame_rate: int) -> List[AudioSegment]:
        audio = AudioSegment.from_raw(
            BytesIO(raw_data), 
            sample_width=sample_width,
            channels=channels,
            frame_rate=frame_rate
        )
        return [segment for segment in chunk_audio(audio)]
    
    def encode_data(self, raw_data: bytes, sample_width: int, channels: int, frame_rate: int):
        parsed = self.parse_audio(raw_data, sample_width, channels, frame_rate)
        parsed = [{'array': np.array(p.get_array_of_samples(), dtype=float)} for p in parsed]
        parsed = [AudioArray.parse_obj(p) for p in parsed]
        return self.preprocess_function(parsed)


model_checkpoint = 'data/panda/wav2vec2-base-finetuned-xvector/best_checkpoint/'
audio_processor = AudioProcessor(model_checkpoint)

files = glob(f'{Path.home()}/.cache/panda/audios/*.wav')
file = choices(files, k=1)[0]

AudioSegment.from_wav(file)

In [ ]:
#| eval: false


from IPython.display import display, Audio


with open(file, "rb") as f:
    audio_bytes = bytearray()
    while (byte := f.read(1)):
        audio_bytes.extend(byte)
encoded_data = audio_processor.encode_data(raw_data=audio_bytes, sample_width=2, channels=2, frame_rate=16000)

for ix, arr in enumerate(encoded_data['input_values']):
    display(Audio(arr.numpy(), rate=32000))


In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()